In [ ]:
!gdown --id 1uV1Fw9QFUofRREiTqCalmq_pUL6R3TjG

Downloading...
From: https://drive.google.com/uc?id=1uV1Fw9QFUofRREiTqCalmq_pUL6R3TjG
To: /content/Dataset.zip
400MB [00:02, 159MB/s]


In [ ]:
!unzip Dataset.zip

Archive:  Dataset.zip
  inflating: SLA_matrix.xlsx         
  inflating: delivery_orders_march.csv  


In [5]:
import pandas as pd
import numpy as np

In [6]:
delivery_df = pd.read_csv('delivery_orders_march.csv')

In [ ]:
delivery_df.shape

(3176313, 6)

In [ ]:
#exclude_date = ['2020-03-01','2020-03-08', '2020-03-15', '2020-03-22', '2020-03-25', '2020-03-29', '2020-03-30', '2020-03-31']

In [ ]:
#locations = ["manila", "luzon", "visayas", "mindanao"]

In [7]:
delivery_df.head(5)

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,1583138397,1.583385e+09,NaN,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,1583309968,1.583463e+09,1.583799e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,1583306434,1.583460e+09,NaN,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,1583419016,1.583556e+09,NaN,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,1583318305,1.583480e+09,NaN,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


# Cleaning the Data

In [8]:
delivery_df.columns

Index(['orderid', 'pick', '1st_deliver_attempt', '2nd_deliver_attempt',
       'buyeraddress', 'selleraddress'],
      dtype='object')

In [9]:
delivery_df.columns = ['orderid', 'pick', 'deliver_attempt_1', 'deliver_attempt_2', 'buyeraddress', 'selleraddress']

In [10]:
delivery_df[['pick', 'deliver_attempt_1', 'deliver_attempt_2']]

,pick,deliver_attempt_1,deliver_attempt_2
0,1583138397,1.583385e+09,NaN
1,1583309968,1.583463e+09,1.583799e+09
2,1583306434,1.583460e+09,NaN
3,1583419016,1.583556e+09,NaN
4,1583318305,1.583480e+09,NaN
...,...,...,...
3176308,1585821728,1.585978e+09,NaN
3176309,1585895011,1.586179e+09,1.586232e+09
3176310,1585826951,1.585965e+09,NaN
3176311,1585891194,1.586230e+09,NaN


In [11]:
# Assuming that the dates provided by the dataset are in UNIX format

def date_parser(date_sr: pd.core.series.Series) -> pd.core.series.Series:
  date_sr = pd.to_datetime(date_sr, unit='s')
  return date_sr

In [12]:
delivery_df['pick'] = date_parser(delivery_df['pick'])
delivery_df['deliver_attempt_1'] = date_parser(delivery_df['deliver_attempt_1'])
delivery_df['deliver_attempt_2'] = date_parser(delivery_df['deliver_attempt_2'])
delivery_df.head(5)

,orderid,pick,deliver_attempt_1,deliver_attempt_2,buyeraddress,selleraddress
0,2215676524,2020-03-02 08:39:57,2020-03-05 05:09:18,NaT,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,2020-03-04 08:19:28,2020-03-06 02:53:56,2020-03-10 00:02:56,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,2020-03-04 07:20:34,2020-03-06 01:56:19,NaT,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,2020-03-05 14:36:56,2020-03-07 04:45:41,NaT,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,2020-03-04 10:38:25,2020-03-06 07:41:40,NaT,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


In [13]:
# Check what datatype is NaT. This is useful for null value checking later.
type(delivery_df['deliver_attempt_2'][0]) == pd._libs.tslibs.nattype.NaTType

True

In [14]:
type(delivery_df)

pandas.core.frame.DataFrame

In [15]:
# Assign the NaT datatype to nat
nat = pd._libs.tslibs.nattype.NaTType

In [16]:
def convert_SellerAddress(dB: pd.core.frame.DataFrame) -> pd.core.series.Series:

    def returnFinalWord(adr: str) -> str:
        proc_ad = adr.split()
        return proc_ad[len(proc_ad)-1].lower()

    address = pd.DataFrame()
    address['raw_address'] = dB['selleraddress']
    address['selleraddress'] = address['raw_address'].apply(lambda x: returnFinalWord(x))

    return address['selleraddress']

In [17]:
def convert_BuyerAddress(dB: pd.core.frame.DataFrame) -> pd.core.series.Series:

    def returnFinalWord(adr: str) -> str:
        proc_ad = adr.split()
        return proc_ad[len(proc_ad)-1].lower()

    address = pd.DataFrame()
    address['raw_address'] = dB['buyeraddress']
    address['buyeraddress'] = address['raw_address'].apply(lambda x: returnFinalWord(x))

    return address['buyeraddress']

In [18]:
delivery_df['buyeraddress'] = convert_BuyerAddress(delivery_df)
delivery_df['selleraddress'] = convert_SellerAddress(delivery_df)

In [19]:
delivery_df.head()

,orderid,pick,deliver_attempt_1,deliver_attempt_2,buyeraddress,selleraddress
0,2215676524,2020-03-02 08:39:57,2020-03-05 05:09:18,NaT,manila,manila
1,2219624609,2020-03-04 08:19:28,2020-03-06 02:53:56,2020-03-10 00:02:56,manila,manila
2,2220979489,2020-03-04 07:20:34,2020-03-06 01:56:19,NaT,manila,manila
3,2221066352,2020-03-05 14:36:56,2020-03-07 04:45:41,NaT,manila,manila
4,2222478803,2020-03-04 10:38:25,2020-03-06 07:41:40,NaT,luzon,manila


# Processing Data

In [20]:
def what_max_day(loc, buyer_address, seller_address):
    max_days = 3
    for x in loc:
        for y in loc:
            temp_max_days = max_days
            if (loc.index(buyer_address) >= loc.index(seller_address)) and (buyer_address == x and seller_address == y):
                return temp_max_days
            elif (loc.index(buyer_address) <= loc.index(seller_address)) and (buyer_address == y and seller_address == x):
                return temp_max_days
            else:
                temp_max_days += 2
                if temp_max_days > 7:
                    temp_max_days = 7
        max_days += 2
        if max_days > 7:
            max_days = 7

In [21]:
locations = ["manila", "luzon", "visayas", "mindanao"]
exclude_date = ['2020-03-01','2020-03-08', '2020-03-15', '2020-03-22', '2020-03-25', '2020-03-29', '2020-03-30', '2020-03-31']
nat = pd._libs.tslibs.nattype.NaTType

In [29]:
def is_late_process(pick, one_deliver_attempt, two_deliver_attempt, buyeraddress, selleraddress):
    #pick = row.pick
    #one_deliver_attempt = row.deliver_attempt_1
    #two_deliver_attempt = row.deliver_attempt_2
    #buyeraddress = row.buyeraddress
    #selleraddress = row.selleraddress
    pick = pd.to_datetime(pick)
    one_deliver_attempt = pd.to_datetime(one_deliver_attempt)
    two_deliver_attempt = pd.to_datetime(two_deliver_attempt)

    #print(type(two_deliver_attempt))
    #print(f"buyeraddress[TYPE]: {type(buyeraddress)}")
    #print(f"selleraddress[TYPE]: {type(selleraddress)}")
    if type(two_deliver_attempt) != nat:
        #print("i went here!!!!!!")
        duration = (two_deliver_attempt - one_deliver_attempt) / np.timedelta64(1, 'D')
        #print(f"BEFORE duration process: \nduration: {duration}, duration[TYPE]: {type(duration)}")
        days_to_exclude = 0
        for i in exclude_date:
            if one_deliver_attempt < pd.to_datetime(i) < two_deliver_attempt:
                days_to_exclude += 1
        duration -= days_to_exclude
        #print(f"AFTER duration process: \nduration: {duration}, duration[TYPE]: {type(duration)}")
    else:
        #print(f"I went here instead lolloololol")
        duration = (one_deliver_attempt - pick) / np.timedelta64(1, 'D')
        #print(duration)
        days_to_exclude = 0 
        for i in exclude_date:
            if pick < pd.to_datetime(i) < one_deliver_attempt:
                days_to_exclude += 1
        duration -= days_to_exclude

    if duration < what_max_day(locations, buyeraddress, selleraddress):
        return 0
    else:
        return 1

In [30]:
result_df = pd.DataFrame()
result_df['orderid'] = delivery_df['orderid']

In [31]:
is_late = np.vectorize(is_late_process)

In [32]:
result_df['is_late'] = is_late(delivery_df['pick'].values, delivery_df['deliver_attempt_1'].values, delivery_df['deliver_attempt_2'], delivery_df['buyeraddress'], delivery_df['selleraddress'])
#result_df['is_late'] = delivery_df.apply(is_late, axis=1)

In [ ]:
duration = (delivery_df['1st_deliver_attempt'].values[0] - delivery_df['pick'].values[0]) / np.timedelta64(1, 'D')

2.853715277777778

In [ ]:
type(delivery_df['deliver_attempt_1'].values[0] - delivery_df['pick'].values[0])

numpy.timedelta64

In [ ]:
delivery_df['pick'][0] < pd.to_datetime(exclude_date[1]) < delivery_df['1st_deliver_attempt'].values[0]

False

In [27]:
delivery_df[delivery_df['orderid'] == 30505877890444]
delivery_df[delivery_df['orderid'] == 2296895366]

,orderid,pick,deliver_attempt_1,deliver_attempt_2,buyeraddress,selleraddress
38937,2296895366,2020-03-12 03:58:08,2020-03-17 01:25:14,2020-03-21 08:34:26,visayas,manila


In [ ]:
delivery_df.shape

(3176313, 6)

In [33]:
result_df.shape

(3176313, 2)

In [34]:
result_df.to_csv('result.csv', index=False)

In [35]:
result_df.head()

,orderid,is_late
0,2215676524,0
1,2219624609,0
2,2220979489,0
3,2221066352,0
4,2222478803,0
